In [4]:
from google.cloud import bigquery
import os
from dotenv import load_dotenv
from IPython.display import display
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd



In [5]:
load_dotenv()

GOOGLE_APPLICATION_CREDENTIALS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
bg_client = bigquery.Client()

SERVICE_ACCOUNT = os.getenv("SERVICE_ACCOUNT")
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT, scopes=SCOPES)
gs_client = gspread.Client(creds)

gmail_messages_dataset_ss = gs_client.open_by_url("https://docs.google.com/spreadsheets/d/1t4BSWD3Wstn0C4iL99h_FZhtIgOviWXFhJn8oMXxcUU/edit?gid=0#gid=0")
worksheet = gmail_messages_dataset_ss.worksheet("all_emails")


In [6]:
sql = """
SELECT
    DISTINCT thread_id
FROM
    `ga4-cenix.email_marketing.gmail_messages`
"""

query_job = bg_client.query(sql)
df_db_thread_id = query_job.to_dataframe()

df_db_thread_id.value_counts()

thread_id       
1.93E+128           1
1992d82fe30274f1    1
1992d98748fc749a    1
1992d987242da5cc    1
1992d987238d0650    1
                   ..
1969fc3f996e5938    1
1969fc3a308aa73e    1
1969fc356a0c1d65    1
1969fc30672081be    1
2.00E+22            1
Name: count, Length: 18907, dtype: int64

In [7]:
df_db_thread_id.describe()

df_db_thread_id

,thread_id
0,199cceeddc34035e
1,1997041e0a9bb76a
2,199cceed9521e1a9
3,19a97ff0c451ead0
4,19a9c8ab36ed4e60
...,...
18902,1994f7470a87c438
18903,1994f74755425816
18904,1995338a78d76730
18905,1995338a7b3876b7


In [8]:
gsheet_thread_id = worksheet.col_values(4)

df_gsheet_thread_id = pd.DataFrame(gsheet_thread_id[1:], columns=[gsheet_thread_id[0]])
df_gsheet_thread_id

,thread_id
0,192d3959124feee8
1,192d3959124feee8
2,192d3ace3b69605b
3,192d3adaea18b95b
4,192dffaf1d86786e
...,...
39534,19a9b7e15f59eae0
39535,19a9b6cbfaa58e93
39536,19a9b4a8d0196957
39537,19a2b4f24e80759f


In [15]:
df_gsheet_thread_id = df_gsheet_thread_id.drop_duplicates()
df_gsheet_thread_id

,thread_id
0,192d3959124feee8
2,192d3ace3b69605b
3,192d3adaea18b95b
4,192dffaf1d86786e
5,192e002d62728ada
...,...
39532,19a9b91b05b4b263
39533,19a9b878cd7d58d9
39534,19a9b7e15f59eae0
39535,19a9b6cbfaa58e93


In [16]:
column_name = "thread_id"

unstored_thread_id = df_gsheet_thread_id[~df_gsheet_thread_id[column_name].isin(df_db_thread_id[column_name])]
unstored_thread_id


,thread_id
30195,1995bac5b87c8d1d
30200,1995bac57d992be1
30202,1995bac55366ea11
30203,1995bac54e84c390
30209,1995bac4f3465a28
30305,19959bd5b9c2957e
30385,19960dfdfc0e92c0
30386,19960dfdf72b68da
30387,19960dfdf31c68e0
30397,19960dfd2ce4c7f6


In [17]:
unstored_thread_id_list = unstored_thread_id.values.tolist()
unstored_thread_final = [x[0] for x in unstored_thread_id_list]
unstored_thread_final

['1995bac5b87c8d1d',
 '1995bac57d992be1',
 '1995bac55366ea11',
 '1995bac54e84c390',
 '1995bac4f3465a28',
 '19959bd5b9c2957e',
 '19960dfdfc0e92c0',
 '19960dfdf72b68da',
 '19960dfdf31c68e0',
 '19960dfd2ce4c7f6',
 '19960dfd22711b01',
 '19960dfb9be02ff2',
 '19960dfb14c8aea7',
 '19960dfac82b807a',
 '19960dfac302ffa7',
 '19962302e99497f7',
 '19960dfcc5b0b632',
 '1996c7dd32cb5c69',
 '19970420d3996b87',
 '19975d01b1d72c61',
 '19975d01a114a37c',
 '19978ab95eb8a899',
 '1997a97b9ac41b69',
 '1997fbe39ffd67dd',
 '1997fbe34b65589a',
 '1997dd216f64d90a',
 '199868059ba68447',
 '1998680592babcb3',
 '19986805310ab005',
 '199868052f2be5c0',
 '2.00E+97',
 '19986805090933d3',
 '19986804ed828ebc',
 '19986804eca266f3',
 '19986165cadd5a02',
 '19986165caae2662',
 '19986165c3feb2e8',
 '19986165a0ce2b38',
 '19986165a05aee8e',
 '199861658e8b4664',
 '199861658b4c781b',
 '1998616586ed44db',
 '199861657d5153a3',
 '199861657866743d',
 '199861656d835c2a',
 '1998616557822cf8',
 '199861655329347a',
 '1998616532599729',


In [12]:
print(unstored_thread_id_list)

[['1995bac5b87c8d1d'], ['1995bac57d992be1'], ['1995bac55366ea11'], ['1995bac54e84c390'], ['1995bac4f3465a28'], ['19959bd5b9c2957e'], ['19960dfdfc0e92c0'], ['19960dfdf72b68da'], ['19960dfdf31c68e0'], ['19960dfd2ce4c7f6'], ['19960dfd22711b01'], ['19960dfb9be02ff2'], ['19960dfb14c8aea7'], ['19960dfac82b807a'], ['19960dfac302ffa7'], ['19962302e99497f7'], ['19960dfcc5b0b632'], ['1996c7dd32cb5c69'], ['19970420d3996b87'], ['19975d01b1d72c61'], ['19975d01a114a37c'], ['19978ab95eb8a899'], ['1997a97b9ac41b69'], ['1997fbe39ffd67dd'], ['1997fbe34b65589a'], ['1997dd216f64d90a'], ['199868059ba68447'], ['1998680592babcb3'], ['19986805310ab005'], ['199868052f2be5c0'], ['2.00E+97'], ['19986805090933d3'], ['19986804ed828ebc'], ['19986804eca266f3'], ['19986165cadd5a02'], ['19986165caae2662'], ['19986165c3feb2e8'], ['19986165a0ce2b38'], ['19986165a05aee8e'], ['199861658e8b4664'], ['199861658b4c781b'], ['1998616586ed44db'], ['199861657d5153a3'], ['199861657866743d'], ['199861656d835c2a'], ['1998616557822cf

In [14]:
sql = """
SELECT
    *
FROM
    `ga4-cenix.email_marketing.gmail_messages`
WHERE
    thread_id='1995bac5b87c8d1d'
"""

query_job = bg_client.query(sql)
df_check = query_job.to_dataframe()

df_check

,uid,timestamp,message_id,thread_id,subject,from_email_address,from_name,to_email_address,to_name,date_sent,text_body,labels,is_agent_message,agent,is_cc
